In [ ]:
import csv

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

def configure_driver():
    #Configurations

     # Đường dẫn đến WebDriver
    webdriver_path = "C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"
    chrome_options = Options()

    # Tắt thông báo Chrome - chặn pop-up
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--disable-notifications") # disable notifications
    # chrome_options.add_argument("--start-maximized") # disable infobars
    chrome_options. add_argument('--blink-settings=imagesEnabled=false') # disable images
    #chrome_options.add_argument('--window-size=800,600')
    
    # Đặt vị trí cửa sổ ở ngoài màn hình chính
    #chrome_options.add_argument('--window-position=2000,0')
    #chrome_options.add_extension("F:\GlobalTech\Stands AdBlocker - Chrome Web Store 2.1.24.0.crx")

    # Sử dụng profile đã có sẵn: 
    # để lưu trạng thái đăng nhập + dùng tool chặn quảng cáo set up sẵn trên Chrome
    # user_profile = "C:\\Users\\21521\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 5"
    # chrome_options.add_argument("user-data-dir=" + user_profile)

    # (Tùy chọn) Chạy Chrome ở chế độ không hiển thị giao diện (chạy ngầm)
    # chrome_options.add_argument("--headless")

    # Khởi tạo dịch vụ và trình duyệt
    service = Service(webdriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver

In [ ]:
# Hàm thu thập liên kết công việc từ một trang
def get_job_links(driver):
    # Tìm tất cả thẻ <a> chứa liên kết công việc
    elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'job_link')]")
    # Lấy thuộc tính 'href' của từng phần tử
    links = [element.get_attribute("href") for element in elements]
    return links


# Hàm duyệt qua nhiều trang và thu thập liên kết
def scrape_multiple_pages(start_url, max_pages):
    driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
    all_links = []

    for page in range(1, max_pages + 1):
        print(f"Đang xử lý trang {page}")
        url = start_url.format(page=page)
        driver.get(url)
        # Thu thập liên kết trên trang hiện tại
        links = get_job_links(driver)
        all_links.extend(links)

    driver.quit()
    return all_links

# URL trang đầu tiên
start_url = "https://careerviet.vn/viec-lam-noi-bat-trong-tuan-l8a30r50p1"

# Thu thập liên kết từ 5 trang đầu tiên
job_links = scrape_multiple_pages(start_url, max_pages=5)

# Lưu kết quả vào CSV
output_file = "job_links.csv"
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Job Link"])
    for link in job_links:
        writer.writerow([link])

print(f"Đã lưu {len(job_links)} liên kết vào {output_file}.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_16052\748314238.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")


Đang xử lý trang 1
Đang xử lý trang 2
Đang xử lý trang 3
Đang xử lý trang 4
Đang xử lý trang 5
Đã lưu 500 liên kết vào job_links.csv.


In [169]:
driver = configure_driver()
url = 'https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh.35C2A242.html'
driver.get(url)

In [170]:
job_detail = {}

# Xử lý cho tab chi tiết công việc

# Lấy tên công việc
job_name = driver.find_element(By.CSS_SELECTOR, "h1[class='title']").text.strip()
job_detail['Tên công việc'] = job_name

job_locations_elements = driver.find_elements(By.CSS_SELECTOR, "div[class='map'] p a")
job_locations = [element.text for element in job_locations_elements]
job_detail['Địa điểm'] = job_locations

# Lấy địa chỉ 
job_elements = driver.find_elements(By.CLASS_NAME, 'has-background')
for element in job_elements:
    li_elements = element.find_elements(By.CSS_SELECTOR, "li")
    for li in li_elements:
        # Lấy tiêu đề từ thẻ <strong>
        label = li.find_element(By.CSS_SELECTOR, "strong").text
        # Lấy giá trị từ thẻ <p>
        value = li.find_element(By.CSS_SELECTOR, "p").text.strip()
        job_detail[label] = value

# Chuyển tab
overview_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@title='Tổng quan công ty']"))
    )
overview_tab.click()

# Xử lý tab thông tin công ty
company_name_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "a[class='name']"))
)
company_name = company_name_element.text.strip()
job_detail['Tên công ty'] = company_name

# company_address = driver.find_element(By.XPATH, ".//div[@class='company-info']//strong[text()='Địa điểm']/following-sibling::text()[1]")
# job_detail['Địa chỉ công ty'] = company_address

# Tìm thẻ cha chứa "Địa điểm"
company_content_element = driver.find_element(By.XPATH, ".//div[@class='company-info']//strong[text()='Địa điểm']/..")

# Lấy toàn bộ nội dung văn bản trong phần tử cha
company_content = company_content_element.text

# Tìm phần địa chỉ công ty
if "Địa điểm" in company_content:
    # Tách lấy phần sau "Địa điểm"
    address_part = company_content.split("Địa điểm", 1)[1]
    
    # Lấy phần trước "Thông tin công ty"
    company_address = address_part.split("Thông tin công ty", 1)[0].strip()
    
    job_detail['Địa chỉ công ty'] = company_address
   

company_li = driver.find_elements(By.CSS_SELECTOR, "div[class='company-info'] ul li")
for li in company_li:
    text = li.text
    if "Người liên hệ" in text:
        job_detail['Người liên hệ'] = text.split(":")[1].strip()
    elif "Qui mô công ty" in text:
        job_detail['Qui mô công ty'] = text.split(":")[1].strip()
    elif "Loại hình hoạt động" in text:
        job_detail['Loại hình hoạt động'] = text.split(":")[1].strip()


In [171]:
job_detail

{'Tên công việc': 'Nhân viên kinh doanh',
 'Địa điểm': ['Hồ Chí Minh', 'Long An'],
 'Ngày cập nhật': '06/12/2024',
 'Ngành nghề': 'Vận chuyển / Giao nhận / Kho vận , Hàng hải , Xuất nhập khẩu',
 'Hình thức': 'Nhân viên chính thức',
 'Lương': 'Cạnh tranh',
 'Kinh nghiệm': '2 - 3 Năm',
 'Cấp bậc': 'Nhân viên',
 'Hết hạn nộp': '28/02/2025',
 'Tên công ty': 'CÔNG TY CỔ PHẦN BOURBON BẾN LỨC',
 'Địa chỉ công ty': 'Lầu 11, P.1107, 29 Lê Duẩn, Q.1',
 'Người liên hệ': 'Ms. Điệp',
 'Qui mô công ty': '100-499',
 'Loại hình hoạt động': 'Cổ phần'}